In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import warnings

import sys
from pyprojroot import here
sys.path.append("../..") 
# from laos_gggi.statistics import get_distance_to_rivers

import pandas as pd
import numpy as np
import pymc as pm
import pytensor
import pytensor.tensor as pt
import matplotlib.pyplot as plt
import arviz as az
import geopandas as gpd
import xarray as xr

from laos_gggi.replication_data import create_replication_data
from laos_gggi.plotting import plotting_function_damages 
from laos_gggi.model import add_data
from laos_gggi.plotting import configure_plot_style

from laos_gggi.sample import sample_or_load
from laos_gggi.transformers import Standardize

from laos_gggi.data_functions import load_emdat_data, load_shapefile, load_rivers_data

In [3]:
# Set random seed
SEED = sum(list(map(ord, 'climate_bayes')))
rng = np.random.default_rng(SEED)

# Loading and preparing data 

In [4]:
# Select SEA countries
laos_neighboors = [
    "KHM",  # Cambodia
    "THA",  # Thailand
    "LAO",  # Laos
    "VNM",  # Vietnam
]

In [6]:
# Configure pytensir floats
floatX = pytensor.config.floatX


# Load idatas
event_idata = az.from_netcdf("model_closest_full.idata")
damage_idata = az.from_netcdf("damages_model.idata")

#Rename ISO dimension to avoid conflicts merging
damage_idata =damage_idata.rename({"ISO": "ISO_damage",
                                                   "country_effect": "country_effect_damage"})

#Merge posteriors
merged_posteriors = xr.merge([event_idata.sel(ISO = laos_neighboors).posterior,
                              damage_idata.posterior])

In [ ]:
# Load maps
world = load_shapefile('world')

# Define maps
sea_map = world.query('ISO_A3 in @laos_neighboors')
laos_map = world.query('ISO_A3 == "LAO"')

### Events data

In [9]:
#Load data sets
sea_point_grid = pd.read_csv("data/sea_point_grid.csv")
laos_point_grid = pd.read_csv("data/laos_point_grid.csv")
sea_df = pd.read_csv("data/sea.csv")
lao_df = pd.read_csv("data/lao.csv")

In [ ]:
#Train transformer
transformer_stand_ =  Standardize().fit(sea_df)

In [ ]:
# Load predictions


In [13]:
# Define time varying features
time_varying_features = ['Population','co2','precip_deviation','dev_ocean_temp','log_population_density',
                            'log_population_density_squared','log_gdp_per_cap','log_gdp_per_cap_squared']
time_varying_features_stand = []

for feature in time_varying_features:
    time_varying_features_stand.append(feature + "__standardized")